In [4]:
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import io
from utils import dl_data

In [6]:
API_URL = "http://0.0.0.0:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=5050): Max retries exceeded with url: /api/v1/login/creds (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff473b95b0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
users = pd.read_csv("../data/csv/API_DATA_DEV - users.csv")
cameras = pd.read_csv("../data/csv/API_DATA_DEV - cameras.csv")
users

In [2]:
user = "test77"
organization_id = users[users["login"]==user]["organization_id"].item()

user_cameras = cameras[cameras["organization_id"]==organization_id]
user_cameras

NameError: name 'users' is not defined

Select some cameras

In [1]:
selected_cameras = [2, 4, 5, 7] # select cameras

In [6]:
if not os.path.isdir("selection-true-positives"):
    dl_data()

In [7]:
sequances_folders = glob.glob("selection-true-positives/*")
len(sequances_folders)

12

## Update heartbeat/last_active_at for selected cameras

In [8]:
# Heartbeat

for camera_id in selected_cameras:
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)

    camera_client.heartbeat()

    sequances_folder = sequances_folders[camera_id]

    

## Update last images for selected cameras

In [9]:
# Update last image

for camera_id in selected_cameras:
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)


    sequances_folder = sequances_folders[camera_id]
    imgs = glob.glob(f"{sequances_folder}/images/*")
    img_file = imgs[0]

    stream = io.BytesIO()
    im = Image.open(img_file)
    im.save(stream, format="JPEG", quality=80)
    response = camera_client.update_last_image(stream.getvalue())
    print(response.text)



    

{"name":"videlles-02","angle_of_view":54.2,"is_trustable":true,"last_image":"2-20250711145724-184321c7.jpg","organization_id":2,"id":2,"elevation":110.0,"lat":48.4783,"lon":2.4242,"last_active_at":"2025-07-11T14:57:24.531372","created_at":"2025-07-11T12:58:08.361140"}
{"name":"croix-augas-02","angle_of_view":54.2,"is_trustable":true,"last_image":"4-20250711145724-a84c05a8.jpg","organization_id":2,"id":4,"elevation":110.0,"lat":48.4267,"lon":2.7109,"last_active_at":"2025-07-11T14:57:24.571574","created_at":"2025-07-11T12:58:08.377063"}
{"name":"moret-sur-loing-01","angle_of_view":54.2,"is_trustable":true,"last_image":"5-20250711145724-d279d770.jpg","organization_id":2,"id":5,"elevation":110.0,"lat":48.3792,"lon":2.8208,"last_active_at":"2025-07-11T14:57:24.617543","created_at":"2025-07-11T12:58:08.404154"}
{"name":"nemours-01","angle_of_view":54.2,"is_trustable":true,"last_image":"7-20250711145724-0c0072b6.jpg","organization_id":2,"id":7,"elevation":110.0,"lat":48.2605,"lon":2.7064,"las